[this doc on github](https://github.com/dotnet/interactive/tree/master/samples/notebooks/polyglot)



In [ ]:
Install-Module -Name Az.ApplicationInsights -RequiredVersion 1.0.3 -Scope CurrentUser 

In [ ]:
Get-PackageProvider

In [ ]:
Connect-AzAccount

In [ ]:
Set-AzContext -Subscription "64276bd9-d4bf-4fe3-8b77-36d696e84b26"

In [ ]:
Install-Module -Name Az.Monitor -Scope CurrentUser

In [ ]:
$logs = Get-AzLog -MaxRecord 200 | Select-Object -ExcludeProperty Authorization,Claims,EventTimestamp,HttpRequest,Level,Properties,SubmissionTimestamp
$logJson = ConvertTo-Json $logs -Depth 3
$logJson | Out-Display -MimeType "application/json"

In [ ]:
$entryByResourceGroup = [Graph.Histogram]::new()
$entryByResourceGroup.name = "By ResourceGroup"
$entryByResourceGroup.x =  $logs.ResourceGroupName

$entryByResourceProvider = [Graph.Histogram]::new()
$entryByResourceProvider.name = "By ResourceProvider"
$entryByResourceProvider.x =  $logs.ResourceProviderName.value

$entryByStatus = [Graph.Histogram]::new()
$entryByStatus.name = "By Status"
$entryByStatus.x =  $logs.Status.value


New-PlotlyChart -Trace @($entryByResourceGroup,$entryByResourceProvider,$entryByStatus) -Title "Events" | Out-Display

In [ ]:
$entryByResourceProviderSuccess = [Graph.Histogram]::new()
$entryByResourceProviderSuccess.name = "Success By ResourceProvider"
$entryByResourceProviderSuccess.x = ($logs | where-object { $_.Status.value -eq "Succeeded"}).ResourceProviderName.value

$entryByResourceProviderFailure = [Graph.Histogram]::new()
$entryByResourceProviderFailure.name = "Failure By ResourceProvider"
$entryByResourceProviderFailure.x =  ($logs | where-object { $_.Status.value -ne "Succeeded"}).ResourceProviderName.value

$layout = [Layout]::new()
$layout.barmode = "stack"
New-PlotlyChart -Layout $layout -Trace @($entryByResourceProviderSuccess, $entryByResourceProviderFailure) -Title "Events outcome by Resource Provider" | Out-Display